
ver1--------------------------------
1.用了自定义的GP_MCMC方法和积分型取平均的EI采集函数
2.和EI_MCMC_random是对标版本
3.去掉了trainningcallback中的交互选项，所以此版本适用于后台运行

ver4----------------------------------
epoch=40，但设置了早停，连续五次没提升就早停
10次随机探索+15次BO


In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
import time
import shutil
import pathlib
import itertools
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
import tensorflow_probability as tfp
tfd = tfp.distributions


from tensorflow import keras
from keras import models, optimizers, metrics, layers, regularizers, losses
from keras.models import Sequential
from keras.optimizers import SGD, RMSprop, Adam, Adagrad
import json

def set_reproducibility(seed=88):
    """
    设置一个全局种子来确保整个实验流程的可复现性。
    这包括Python内置random、os、NumPy和TensorFlow。
    同时，它会尝试配置TensorFlow使用确定性算法。
    
    Args:
        seed (int): 您选择的随机种子。
    """
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    tf.config.experimental.enable_op_determinism()
    
    print(f"✅ 全局随机种子已设置为: {seed}，并已启用TensorFlow确定性操作。")


SEED_VALUE = 88 
set_reproducibility(SEED_VALUE)

In [ ]:
#数据集会自动解压缩，只需要直接讲数据加载到output中

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
from tensorflow import keras
from keras import layers
import matplotlib.image as img


'''加载到output路径并划分文件夹'''

!pip install split-folders -q
import splitfolders
input_dir="/kaggle/input/augmented-alzheimer-data/augmented_data"
output_dir="./output"
splitfolders.ratio(
    input_dir, 
    output=output_dir, 
    seed=1345, 
    ratio=(.70, 0.15,0.15),
    group_prefix=None 
) 
print("Dataset split completed!")

#验证输出路径文件夹文件数量
def count_files(directory):
    for split in ['train', 'val', 'test']:
        split_path = os.path.join(directory, split)
        print(f"\n{split.upper()} set:")
        for class_name in os.listdir(split_path):
            class_path = os.path.join(split_path, class_name)
            num_files = len(os.listdir(class_path))
            print(f"  {class_name}: {num_files} images")

count_files("./output")

AUTOTUNE = tf.data.AUTOTUNE
IMG_HEIGHT = 224
IMG_WIDTH = 224
batch_size=32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
"./output/train",
seed=123,
label_mode='int',#这是默认值
image_size=(IMG_HEIGHT, IMG_WIDTH),
batch_size=batch_size,
shuffle=True
).prefetch(buffer_size=AUTOTUNE)
# 默认情况下label_mode='int'后续匹配sparse_categorical_crossentropy	需设置from_logits=True/False


test_ds = tf.keras.preprocessing.image_dataset_from_directory(
"./output/test",
label_mode='int',
seed=123,
image_size=(IMG_HEIGHT, IMG_WIDTH),
batch_size=batch_size,
shuffle=False  
).prefetch(buffer_size=AUTOTUNE)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
"./output/val",
label_mode='int',
seed=123,
image_size=(IMG_HEIGHT, IMG_WIDTH),
batch_size=batch_size,
shuffle=False
).prefetch(buffer_size=AUTOTUNE)

In [ ]:
import os, random, time, gc, json
import numpy as np, pandas as pd, seaborn as sns, matplotlib.pyplot as plt
import tensorflow as tf, tensorflow_probability as tfp
from tensorflow import keras
from keras import layers, regularizers
import scipy.stats as stats
tfd = tfp.distributions
sns.set_style('whitegrid')
tf.keras.backend.set_floatx('float64')

#-----------------------------模型架构搭建：顺序模型------------------------------

def build_pretrained_model(hparams):
    # 1. 加载预训练的EfficientNet，不包含顶部的分类层
    base_model = tf.keras.applications.EfficientNetB0(
        input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
        include_top=False,
        weights='imagenet'
    )
    # 2. 冻结预训练模型的权重
    base_model.trainable = False
    
    # 3. 确定从哪个块开始解冻（微调）
    fine_tune_at_block = 'block5a_expand_conv'
    # 3. 遍历所有层，解冻 fine_tune_at_block 及之后的所有层
    set_trainable = False
    for layer in base_model.layers:
        if layer.name == fine_tune_at_block:
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False

    # 最佳实践：无论是否解冻，都建议保持 BatchNormalization 层处于冻结状态
    # 在微调时，用小批量数据更新BN层的统计数据可能会引入噪声，破坏预训练学到的分布
    for layer in base_model.layers:
        if isinstance(layer, layers.BatchNormalization):
            layer.trainable = False
    
    # # --- 核心修改 1: 解冻模型的最后20层 (在微调时，通常保持 BatchNormalization 层被冻结，这有助于稳定训练)
    # for layer in base_model.layers[-20:]:
    #     if not isinstance(layer, layers.BatchNormalization):
    #         layer.trainable = True
            
    dense_units = int(hparams.get('dense_units', 128))
    dropout_rate = hparams.get('dropout_rate', 0.5)
    l2_reg = hparams.get('l2_reg', 0.001)
    lr = hparams.get('lr', 0.0001)
    optimizer_choice = int(hparams.get('optimizer_choice', 0))
    momentum = hparams.get('momentum', 0.9)
    
    


    # 3. 在预训练模型之上，构建一个【小而精】的分类头
    model = tf.keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(), # 使用全局平均池化替代Flatten，参数更少，不易过拟合
        layers.Dense(dense_units, activation='relu', kernel_regularizer=regularizers.l2(l2_reg)), # 使用更小的L2值
        layers.Dropout(dropout_rate),
        layers.Dense(4, activation='softmax') # 4个类别
    ])

 
    if optimizer_choice == 0:
        print("--- 使用 Adam 优化器 ---")
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    elif optimizer_choice == 1:
        print(f"--- 使用 SGD 优化器 (momentum={momentum:.3f}) ---")
        optimizer = tf.keras.optimizers.SGD(learning_rate=lr, momentum=momentum)
    elif optimizer_choice == 2:
        print("--- 使用 RMSprop 优化器 ---")
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=lr)
    elif optimizer_choice ==3:
        print("--- 使用 Adagrad 优化器 ---")
        optimizer = tf.keras.optimizers.Adagrad(learning_rate=lr)
    else: # optimizer_choice == 分数
        print("--- 无法找到对应的优化器，请检查optimizer_choice value ---")
        

    
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
   
    return model



#---------------------------------------------------------------------------------------------------------
class TrainingCallback(keras.callbacks.Callback):
    def __init__(self, epochs):
        super().__init__()
        self.epochs = epochs
    def on_train_begin(self, logs=None):
        header = "{0:^8s}{1:^11s}{2:^11s}{3:^12s}{4:^12s}{5:^12s}".format('Epoch', 'Loss', 'Acc', 'Val_Loss', 'Val_Acc', 'Duration')
        print(header)
    def on_epoch_begin(self, epoch, logs=None): 
        self.ep_start = time.time()
    def on_epoch_end(self, epoch, logs=None):
        duration = time.time() - self.ep_start
        acc = logs.get('accuracy', 0); val_acc = logs.get('val_accuracy', 0)
        loss = logs.get('loss', 0); val_loss = logs.get('val_loss', 0)
        msg = f'{str(epoch + 1):^3s}/{str(self.epochs):4s} {loss:^10.3f}{acc * 100:^11.3f}{val_loss:^12.5f}{val_acc * 100:^12.3f}{duration:^12.2f}'
        print(msg)
        

#--------------------------------------------------------------------------------------------------------


def train_and_evaluate_model(hparams,train_ds,val_ds,epochs=30):
    #创建模型
    model = build_pretrained_model(hparams) 
    

    training_callback = TrainingCallback(
        epochs=epochs
    )
    
    early_stopping_callback = tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=5,  # 验证集准确率连续5个epoch没有提升就停止
        min_delta=0.005,
        restore_best_weights=True, # 关键：自动恢复到性能最好的那个epoch的权重
        verbose=0 # 在BO循环中保持安静
    )
    
    #训练模型
    history = model.fit(
        train_ds,#指定训练数据集(必需参数)
         #验证数据集(可选参数)在每个epoch结束后，模型会在这个数据集上评估性能
        validation_data=val_ds,
        epochs=epochs,#遍历训练数据的总轮次(必需参数)
        verbose=0,#日志显示模式,设置为0时,通常是因为使用了自定义回调函数来显示更详细的信息
        #batch_size=batch_size,
        callbacks=[training_callback,early_stopping_callback]
    )

    # 返回验证集准确率（贝叶斯优化会最大化此值）
    loss, accuracy = model.evaluate(val_ds, verbose=0)
    # 我们可以从早停回调中获取最佳轮次信息
    best_epoch_num = early_stopping_callback.best_epoch + 1
    print(f"  > 训练评估完成: Best Val Acc: {accuracy:.4f}, 对应的 Loss:{loss:.4f}(at epoch {best_epoch_num})")
    
    return accuracy
    


In [ ]:
class BayesianOptimizer:
    def __init__(self, search_space, init_points=5, exploration=0.01, 
                 log_file="custom_bo_log.json", initial_X=None, initial_y=None):
        self.search_space = np.array(search_space)
        self.dim = len(search_space)
        self.init_points = init_points
        self.exploration = exploration
        self.log_file = log_file
        #----以防训练由于超时中断,设置自断点处接着BO迭代,加载已计算好的数据
        if initial_X is not None and initial_y is not None:
            self.X_obs = initial_X
            self.y_obs = initial_y
            print(f"BO optimizer has used {len(self.y_obs)} observed datas to initialize")
            #日志回填
            print(f"new log '{self.log_file}' is set and loads old data into it ")
            if os.path.exists(self.log_file):
                os.remove(self.log_file)
            for i in range(len(self.y_obs)):
                self._write_log(iteration=i + 1, x_new=self.X_obs[i], y_new=self.y_obs[i])
            print(f"历史数据回填完成。共写入 {len(self.y_obs)} 条记录。")
        else: 
            self.X_obs = np.zeros((0, self.dim))
            self.y_obs = np.zeros(0)
            if os.path.exists(self.log_file):
                os.remove(self.log_file)
                
        self.theta_dim = self.dim + 2 # length_scales (dim) + amplitude + noise
        
        self.num_samples = 20
        self.burn_in = 100
        
    
        
    def random_sample(self):
        """在搜索空间内随机采样一个超参数点。"""
        return np.array([np.random.uniform(low, high) for (low, high) in self.search_space])

    @tf.function(jit_compile=True) 
    def matern52_kernel(self, X1, X2, theta):
        X1_tf = tf.cast(X1, tf.float64)
        X2_tf = tf.cast(X2, tf.float64)
        theta = tf.cast(theta, tf.float64)
        
        length_scales = theta[:self.dim]
        amplitude = theta[self.dim]
        
        X1_scaled = X1_tf / length_scales
        X2_scaled = X2_tf / length_scales

        
        r2 = tf.reduce_sum(tf.square(X1_scaled), axis=1, keepdims=True) - 2 * tf.matmul(X1_scaled, X2_scaled, transpose_b=True) + tf.reduce_sum(tf.square(X2_scaled), axis=1)
        r2 = tf.maximum(r2, 1e-12)
        r = tf.sqrt(r2)
        
        # --- 关键修改 ---
        # 将所有Python字面量常数转换为tf.float64类型的张量
        five_64 = tf.constant(5.0, dtype=tf.float64)
        three_64 = tf.constant(3.0, dtype=tf.float64)
        one_64 = tf.constant(1.0, dtype=tf.float64)
        
        sqrt5_64 = tf.sqrt(five_64)
        
        # 使用我们新定义的tf.float64常数进行计算
        term = (one_64 + sqrt5_64 * r + (five_64 / three_64) * r2) * tf.exp(-sqrt5_64 * r)
        
        K = amplitude**2 * term
        return K

    
    @tf.function(jit_compile=True) 
    def log_posterior(self, theta, X, y): 
        f64 = tf.constant(1e-6, dtype=tf.float64)
        two_pi_64 = tf.constant(2.0 * np.pi, dtype=tf.float64)
        half_64 = tf.constant(0.5, dtype=tf.float64)
        
        X_tf = tf.cast(X, dtype=tf.float64)
        y_tf = tf.cast(y, dtype=tf.float64)
        theta_tf = tf.cast(theta, dtype=tf.float64)
        
        noise = theta_tf[self.dim + 1]**2
        
        K = self.matern52_kernel(X_tf, X_tf, theta_tf)
        n = tf.shape(K)[0]
        K += (noise + f64) * tf.eye(n, dtype=tf.float64)
        
        L = tf.linalg.cholesky(K)
        alpha = tf.linalg.cholesky_solve(L, tf.expand_dims(y_tf, 1))
        
        log_lik = -half_64 * tf.squeeze(tf.matmul(tf.expand_dims(y_tf, 0), alpha))
        log_lik -= tf.reduce_sum(tf.math.log(tf.linalg.diag_part(L)))
        n_64 = tf.cast(n, tf.float64)
        log_lik -= half_64 * n_64 * tf.math.log(two_pi_64)
        
        return log_lik


    def sample_theta_posterior(self):
        def target_log_prob_fn(theta):
            return self.log_posterior(theta, self.X_obs, self.y_obs)

        kernel = tfp.mcmc.HamiltonianMonteCarlo(
            target_log_prob_fn=target_log_prob_fn,
            num_leapfrog_steps=3,
            step_size=0.1)
        
        samples = tfp.mcmc.sample_chain(
            num_results=self.num_samples,
            current_state=tf.ones(self.theta_dim, dtype=tf.float64),
            kernel=kernel,
            num_burnin_steps=self.burn_in,
            trace_fn=None)
        
        return samples.numpy()
    
    def expected_improvement(self, x_candidate, theta_samples):
        if self.y_obs.size == 0: return 1.0
        
        y_best = np.max(self.y_obs)
        x_candidate_reshaped = x_candidate.reshape(1, -1)
        
        ei_values = []
        for theta in theta_samples:

            try:
                theta_np = theta.numpy() if hasattr(theta, 'numpy') else theta
                
                K_obs = self.matern52_kernel(self.X_obs, self.X_obs, theta_np).numpy()
                noise = theta_np[self.dim + 1]**2
                K_obs += (noise + 1e-6) * np.eye(len(self.X_obs))
                
                K_cross = self.matern52_kernel(self.X_obs, x_candidate_reshaped, theta_np).numpy()
                K_candidate = self.matern52_kernel(x_candidate_reshaped, x_candidate_reshaped, theta_np).numpy()
                L = np.linalg.cholesky(K_obs)
                L_inv_K_cross = np.linalg.solve(L, K_cross)
                
                mu = np.dot(L_inv_K_cross.T, np.linalg.solve(L, self.y_obs))
                sigma2 = K_candidate - np.dot(L_inv_K_cross.T, L_inv_K_cross)

                # --- 核心修改：安全地将 mu 和 sigma2 转换为标量 ---
                # mu 是 (1,) 形状的一维数组, 用 [0] 提取
                mu_scalar = mu[0]
                # sigma2 是 (1,1) 形状的二维数组, 用 [0,0] 提取
                sigma2_scalar = sigma2[0, 0]
                
                sigma = np.sqrt(np.maximum(sigma2_scalar, 1e-9))
                # 如果 sigma 过小，则此点没有不确定性，EI为0
                if sigma == 0.0:
                    continue

                z = (mu_scalar - y_best - self.exploration) / sigma
                
                norm = stats.norm(loc=0, scale=1)
                ei = (mu_scalar - y_best - self.exploration) * norm.cdf(z) + sigma * norm.pdf(z)
                
                # 确保 ei 是一个正常的浮点数
                if not np.isnan(ei):
                    ei_values.append(ei)

            except np.linalg.LinAlgError:
                # 矩阵计算出错时跳过此样本
                print("警告: 在EI计算中发生矩阵奇异值错误，跳过此theta样本。")
                continue
                
        return np.mean(ei_values) if ei_values else 0.0
         

    
    def _write_log(self, iteration, x_new, y_new):
        param_names = ['lr','dense_units','dropout_rate','l2_reg','optimizer_choice','momentum']
        params_dict = {name:val for name, val in zip(param_names, x_new)}
        log_entry={'iteration':int(iteration),
                   'accuracy':float(y_new),
                   'params':params_dict}
        with open(self.log_file,'a') as f: 
            f.write(json.dumps(log_entry)+'\n')
    
    def optimize(self, objective_fn, n_iter=30):
        #检查是 初次BO 还是 接续BO
        num_existing_points=self.X_obs.shape[0]
        num_init_points_to_run = max(0, self.init_points - num_existing_points)

        if num_init_points_to_run > 0:
            # 场景A: 加载的点数少于要求的初始随机化点数，需要补充运行
            print(f"--- 继续随机初始化阶段: 还需要运行 {num_init_points_to_run} 个初始点。 ---")
            for i in range(num_init_points_to_run):
                print(f"\n--- 初始点 {num_existing_points + i + 1}/{self.init_points} ---")
                x_new = self.random_sample()
                y_new = objective_fn(x_new)
                self.X_obs = np.vstack([self.X_obs, x_new])
                self.y_obs = np.append(self.y_obs, y_new)
                self._write_log(iteration=num_existing_points + i + 1, x_new=x_new, y_new=y_new)
        
        elif num_existing_points > 0:
            # 场景B: 成功恢复运行。加载的点数已满足要求，跳过随机搜索。
            print(f"--- 成功恢复优化: 已加载 {num_existing_points} 个点。跳过 {self.init_points} 个初始点的随机搜索阶段。 ---")
        
        else:
            # 场景C: 全新运行 (没有加载任何数据)
            print(f"--- 开始全新运行: 执行 {self.init_points} 个初始随机点 ---")
            for i in range(self.init_points):
                print(f"\n--- 初始点 {i + 1}/{self.init_points} ---")
                x_new = self.random_sample()
                y_new = objective_fn(x_new)
                self.X_obs = np.vstack([self.X_obs, x_new])
                self.y_obs = np.append(self.y_obs, y_new)
                self._write_log(iteration=i + 1, x_new=x_new, y_new=y_new)

        print(f"\n--- 开始执行 {n_iter} 次新的贝叶斯优化迭代 ---")

        
        for i in range(n_iter):
            print(f"\n--- 优化迭代 {i+1}/{n_iter} ---")
            
            theta_samples = self.sample_theta_posterior()
            
            best_ei = -1
            best_x = None
            for _ in range(200): # 增加随机搜索次数以更好地优化采集函数
                x_candidate = self.random_sample()
                ei = self.expected_improvement(x_candidate, theta_samples)
                if ei > best_ei:
                    best_ei = ei
                    best_x = x_candidate
            
            if best_x is None:
                print("警告：无法找到有效的候选点，将进行随机采样。")
                best_x = self.random_sample()

            y_new = objective_fn(best_x)
            
            self.X_obs = np.vstack([self.X_obs, best_x])
            self.y_obs = np.append(self.y_obs, y_new)
            # 在每次优化后也写入日志 ---
            self._write_log(iteration=self.init_points + i + 1, x_new=best_x, y_new=y_new)


        best_idx = np.argmax(self.y_obs)
        return self.X_obs[best_idx], self.y_obs[best_idx]

   
            


In [ ]:
import time
import gc 
#---------------------------- main---------------------------------

search_space = [
    (1e-5, 5e-4),    # lr
    (64, 512),       # dense_units
    (0.2, 0.7),      # dropout_rate
    (1e-5, 5e-3),    # l2_reg
    (0, 3.99),      # 0=Adam, 1=SGD, 2=RMSprop, 3=Adagrad
    (0.85, 0.99)     # momentum
]
#=============================================================================================
def my_objective_fn(x_new):
    # 将优化器给出的numpy数组转换为超参数字典
    hparams = {
        'lr': x_new[0],
        'dense_units': int(x_new[1]),
        'dropout_rate': x_new[2],
        'l2_reg': x_new[3],
        'optimizer_choice': int(x_new[4]),
        'momentum': x_new[5],
    }
    # 清理会话并调用训练函数
    tf.keras.backend.clear_session()
    gc.collect()
    print(f"\n正在评估超参数: {hparams}")
    accuracy = train_and_evaluate_model(hparams, train_ds, val_ds, epochs=40)
    print(f"评估完成 - 验证集准确率: {accuracy:.4f}")
    return accuracy

# 4.4 实例化并运行优化器
optimizer = BayesianOptimizer(search_space=search_space, 
                              init_points=10,
                              log_file="./EI_MCMC_BO_log.json" 
                             )

best_x, best_y = optimizer.optimize(objective_fn=my_objective_fn, n_iter=15)  
#===============================================================================================

print("\n" + "="*50); print("贝叶斯优化完成!")
print(f"最佳验证准确率: {best_y:.4f}")

param_names = ['lr', 'dense_units', 'dropout_rate', 'l2_reg', 'optimizer_choice', 'momentum']
best_hparams = {name: val for name, val in zip(param_names, best_x)}
# 修正类型
best_hparams['dense_units'] = int(best_hparams['dense_units'])
best_hparams['optimizer_choice'] = int(best_hparams['optimizer_choice'])

print(f"最佳超参数: {best_hparams}")
print("="*50)

In [ ]:
# ===================================================================
# 贝叶斯优化全过程的图像表示
# ===================================================================

import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


sns.set_style('whitegrid')

log_file_path = "EI_MCMC_BO_log.json" 

initial_points = 10

try:
    with open(log_file_path, 'r') as f:
        logs = [json.loads(line) for line in f]
    
    df_results = pd.DataFrame(logs)
    all_accuracies = df_results['accuracy'].values
    print(f"✅ Log file '{log_file_path}' loaded successfully! Loaded {len(all_accuracies)} evaluation records.")
    
    
    if len(all_accuracies) > initial_points:
        accuracies_in_init_phase = all_accuracies[:initial_points]
        best_random_accuracy = np.max(accuracies_in_init_phase)
        best_final_accuracy = np.max(all_accuracies)
        absolute_improvement = best_final_accuracy - best_random_accuracy
        
        print("\n" + "="*50)
        print("最终优化数据:")
        print("="*50)
        print(f"- 全局最优Val Accuracy:                 {best_final_accuracy:.4f}")
        print(f"- 探索阶段的最优Val Accuracy :  {best_random_accuracy:.4f}")
        
        if best_random_accuracy > 0:
            relative_improvement = (absolute_improvement / best_random_accuracy) * 100
            print(f"- Absolute Improvement from Bayesian Opt: +{absolute_improvement:.4f} points")
            print(f"- Relative Improvement from Bayesian Opt: +{relative_improvement:.2f}%")
        else:
            print(f"- Absolute Improvement from Bayesian Opt: +{absolute_improvement:.4f} points")
        print("="*50)

    else:
        print("\nInsufficient evaluations to calculate Bayesian optimization improvement.")

   
    n_evaluations = len(all_accuracies)
    x_axis = np.arange(1, n_evaluations + 1)
    best_idx = np.argmax(all_accuracies)
    best_x = best_idx + 1
    best_y = all_accuracies[best_idx]
    plt.figure(figsize=(14, 8))
  
    plt.plot(x_axis, all_accuracies, 'o-', label='Evaluation Accuracy', color='steelblue', alpha=0.7)

    plt.plot(x_axis, np.maximum.accumulate(all_accuracies), 'r--', label='Progressive Best Accuracy', linewidth=2.5)

    plt.plot(best_x, best_y, 
             marker='*', 
             color='gold', 
             markersize=25, 
             markeredgecolor='black',
             linestyle='', 
             label=f'Global Best Point (Accuracy: {best_y:.4f})\nat Evaluation #{best_x}')

   
    plt.axvline(initial_points, color='gray', linestyle=':', linewidth=2, label=f'End of {initial_points} Random Explorations')

   
    plt.xlabel('Number of Evaluations', fontsize=12)
    plt.ylabel('Validation Accuracy', fontsize=12)
    plt.title(f'Bayesian Optimization Progress Curve (Overall Best Accuracy: {best_final_accuracy:.4f})', fontsize=16)
    plt.grid(True, which="both", ls="--", alpha=0.6)
    plt.legend(fontsize=12)
    plt.tight_layout()

    
    plt.savefig('final_optimization_curve_en.png')
    plt.show()

except FileNotFoundError:
    print(f"Error: Log file '{log_file_path}' not found. Please ensure the path is correct.")
except Exception as e:
    print(f"An error occurred during processing or plotting: {e}")